In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np



## Dados Reais

### Manipulação dos dados

In [ ]:
# Dataset dados reais

df1 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Exp_00/main/df1_trat.csv')# falha de subcarga
df2 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Exp_00/main/df2_trat.csv') # sem falha de subcarga

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11688 entries, 0 to 11687
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Datetime              11688 non-null  object 
 1   RTU_COMP_WATT         11688 non-null  float64
 2   RTU_LA_COND_TEMP      11688 non-null  float64
 3   RTU_MA_HUM            11688 non-null  float64
 4   RTU_MA_TEMP           11688 non-null  float64
 5   RTU_OA_HUM            11688 non-null  float64
 6   RTU_OA_TEMP           11688 non-null  float64
 7   RTU_RA_HUM            11688 non-null  float64
 8   RTU_RA_TEMP           11688 non-null  float64
 9   RTU_REFG_COND_TEMP_1  11688 non-null  float64
 10  RTU_REFG_COND_TEMP_2  11688 non-null  float64
 11  RTU_REFG_DISC_PRES_1  11688 non-null  float64
 12  RTU_REFG_DISC_PRES_2  11688 non-null  float64
 13  RTU_REFG_DISC_TEMP_1  11688 non-null  float64
 14  RTU_REFG_DISC_TEMP_2  11688 non-null  float64
 15  RTU_REFG_SUCT_PRES_

In [ ]:
columstodrop = ['RTU_LA_COND_TEMP','RTU_MA_HUM','RTU_MA_TEMP','RTU_REFG_DISC_TEMP_1','RTU_REFG_SUCT_TEMP_1','RTU_REFG_COND_TEMP_1']
#colunas que não há no dado simulado as 'REFG' foram eliminadas pq só o segundo momento será aceito.


In [ ]:
dados_combinados = pd.concat([df1, df2], ignore_index=True)


In [ ]:
dados_combinados = dados_combinados.drop(columstodrop, axis=1)


In [ ]:
dados_combinados['RTU_REFG_DISC_PRES'] = dados_combinados['RTU_REFG_DISC_PRES_1'] + dados_combinados['RTU_REFG_DISC_PRES_2']

In [ ]:
dados_combinados['RTU_REFG_SUCT_PRES'] = dados_combinados['RTU_REFG_SUCT_PRES_1'] + dados_combinados['RTU_REFG_SUCT_PRES_2']

In [ ]:
columstodrop2 = ['RTU_REFG_DISC_PRES_1','RTU_REFG_DISC_PRES_2','RTU_REFG_SUCT_PRES_1','RTU_REFG_SUCT_PRES_2']

In [ ]:
dados_combinados = dados_combinados.drop(columstodrop2, axis = 1)

In [ ]:
novo_nome_colunas = {
    'RTU_REFG_COND_TEMP_2': 'RTU_REFG_COND_TEMP',
    'RTU_REFG_DISC_TEMP_2': 'RTU_REFG_DISC_TEMP',
    'RTU_REFG_SUCT_TEMP_2':'RTU_REFG_SUCT_TEMP'}

In [ ]:
dados_combinados = dados_combinados.rename(columns=novo_nome_colunas)

#### Derivada

In [ ]:
# Calcular a Derivada da Pressão (N-1) por Minuto

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
dados_combinados['Datetime'] = pd.to_datetime(dados_combinados['Datetime'])

# Ordene o DataFrame pela coluna 'DataHora' (caso não esteja ordenado)
dados_combinados.sort_values(by='Datetime', inplace=True)




# Calcule a derivada por minuto para 'RTU_REFG_DISC_PRES'

dados_combinados['Derivada_RTU_REFG_DISC_PRES'] = dados_combinados['RTU_REFG_DISC_PRES'].diff() / dados_combinados['Datetime'].diff().dt.total_seconds()

# Calcule a derivada por minuto para 'RTU_REFG_SUCT_PRES'

dados_combinados['Derivada_RTU_REFG_SUCT_PRES'] = dados_combinados['RTU_REFG_SUCT_PRES'].diff() / dados_combinados['Datetime'].diff().dt.total_seconds()



In [ ]:
# Calcular a Média móvel


janela_media_movel = 1440   # Janela de um dia (1440 minutos)

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'

dados_combinados['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = dados_combinados['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'

dados_combinados['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = dados_combinados['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()



In [ ]:
# Calcular o Desvio Padrão


janela_um_dia = 1440 # Defina a janela de 1 dia (1440 minutos)


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
dados_combinados['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = dados_combinados['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'

dados_combinados['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = dados_combinados['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()



In [ ]:
# Padronizar Manualmente

# Calcule o Z-Score para cada minuto usando a média e o desvio padrão das colunas
dados_combinados['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (dados_combinados['Derivada_RTU_REFG_SUCT_PRES'] - dados_combinados['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / dados_combinados['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']


dados_combinados['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (dados_combinados['Derivada_RTU_REFG_DISC_PRES'] - dados_combinados['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / dados_combinados['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']



In [ ]:
# eliminar os valores iniciais Nulos  das primeiras 24 hrs

dados_combinados = dados_combinados.dropna()

#### dados que serão trabalhados

In [ ]:
#  matriz features sem derivada

X1 = dados_combinados.drop(["Target","Datetime", 'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES'], axis =1)
y1 = dados_combinados['Target']

In [ ]:
# Matriz  com feature derivada Pura

X2 = dados_combinados.drop(["Target","Datetime", 'RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES'], axis =1)
y2 = dados_combinados['Target']


In [ ]:
# Matriz com  derivada Média

In [ ]:
X3 = dados_combinados.drop(["Target","Datetime", 'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES','RTU_REFG_DISC_PRES','RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES'], axis =1)
y3 = dados_combinados['Target']

In [ ]:
# Matriz com  derivada Normalizada

X4 = dados_combinados.drop(["Target","Datetime",'RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES',
    'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'], axis =1)
y4 = dados_combinados['Target']

#### Normalização

In [ ]:
from sklearn.preprocessing import StandardScaler


# Crie um objeto StandardScaler
scalerR = StandardScaler()

# Ajuste e transforme os dados
X1 = scalerR.fit_transform(X1)
X2 = scalerR.fit_transform(X2)
X3 = scalerR.fit_transform(X3)
X4 = scalerR.fit_transform(X4)

#### Separação treino e teste


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, stratify=y1,random_state= 27) # dados sem derivada

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, stratify=y2,random_state= 27) # dados com derivada pura

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, stratify=y3,random_state= 27) # dados com derivada média

X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.2, stratify=y4,random_state= 27) # dados com derivada normalizada



## Dados Simulados

### Manipulação dados simuladas

In [ ]:
# Dataset dados simulados

df20 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df20.csv') # falha de subcarga 20%
dfbs = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/dfbs.csv') #  sem de falha de subcarga

In [ ]:
coluna_a_manter = ['Datetime','RTU_COMP_WATT', 'RTU_OA_HUM', 'RTU_OA_TEMP', 'RTU_RA_HUM',
                   'RTU_RA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_DISC_TEMP', 'RTU_REFG_SUCT_TEMP',
                   'RTU_SA_FAN_WATT', 'RTU_SA_FLOW', 'RTU_SA_HUM', 'RTU_SA_TEMP',
                   'RTU_TOT_WATT', 'ZA_HUM', 'ZA_TEMP', 'RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES']

dfsimf = df20[coluna_a_manter].assign(Target = 1) # falha de subcarga

dfsimbs = dfbs[coluna_a_manter].assign(Target = 0) #  não falha

In [ ]:
dfsimf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143941 entries, 0 to 143940
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Datetime            143941 non-null  object 
 1   RTU_COMP_WATT       143941 non-null  float64
 2   RTU_OA_HUM          143941 non-null  float64
 3   RTU_OA_TEMP         143941 non-null  float64
 4   RTU_RA_HUM          143941 non-null  float64
 5   RTU_RA_TEMP         143941 non-null  float64
 6   RTU_REFG_COND_TEMP  143941 non-null  float64
 7   RTU_REFG_DISC_TEMP  143941 non-null  float64
 8   RTU_REFG_SUCT_TEMP  143941 non-null  float64
 9   RTU_SA_FAN_WATT     143941 non-null  float64
 10  RTU_SA_FLOW         143941 non-null  float64
 11  RTU_SA_HUM          143941 non-null  float64
 12  RTU_SA_TEMP         143941 non-null  float64
 13  RTU_TOT_WATT        143941 non-null  float64
 14  ZA_HUM              143941 non-null  float64
 15  ZA_TEMP             143941 non-nul

In [ ]:
dados_combinadossim = pd.concat([dfsimf, dfsimbs], ignore_index=True)


In [ ]:
# Copie as colunas originais antes da conversão
dados_combinadossim_copy = dados_combinadossim.copy()

# Columns to convert - Temperatura
temperature_columns_to_convert = ['RTU_OA_TEMP', 'RTU_RA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_DISC_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_SA_TEMP', 'ZA_TEMP']

# Conversion function from Fahrenheit to Celsius
def fahrenheit_to_celsius(fahrenheit):
    return (fahrenheit - 32) * 5/9

# Apply the conversion to the specified temperature columns
dados_combinadossim[temperature_columns_to_convert] = dados_combinadossim[temperature_columns_to_convert].apply(fahrenheit_to_celsius)

# Columns to convert - Pressão
pressure_columns_to_convert = ['RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES']

# Conversion function from Pascal to Bar
def pascal_to_bar(pascal):
    return pascal * 1e-5

# Apply the conversion to the specified pressure columns
dados_combinadossim[pressure_columns_to_convert] = dados_combinadossim[pressure_columns_to_convert].apply(pascal_to_bar)


#### Derivada

In [ ]:
# Calcular a Derivada da Pressão (N-1) por Minuto

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
dados_combinadossim['Datetime'] = pd.to_datetime(dados_combinadossim['Datetime'])

# Calcule a derivada por minuto para 'RTU_REFG_DISC_PRES'

dados_combinadossim['Derivada_RTU_REFG_DISC_PRES'] = dados_combinadossim['RTU_REFG_DISC_PRES'].diff() / dados_combinadossim['Datetime'].diff().dt.total_seconds()

# Calcule a derivada por minuto para 'RTU_REFG_SUCT_PRES'

dados_combinadossim['Derivada_RTU_REFG_SUCT_PRES'] = dados_combinadossim['RTU_REFG_SUCT_PRES'].diff() / dados_combinadossim['Datetime'].diff().dt.total_seconds()



In [ ]:
dados_combinadossim.head(200)

,Datetime,RTU_COMP_WATT,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_HUM,RTU_RA_TEMP,RTU_REFG_COND_TEMP,RTU_REFG_DISC_TEMP,RTU_REFG_SUCT_TEMP,RTU_SA_FAN_WATT,...,RTU_SA_HUM,RTU_SA_TEMP,RTU_TOT_WATT,ZA_HUM,ZA_TEMP,RTU_REFG_DISC_PRES,RTU_REFG_SUCT_PRES,Target,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES
0,2018-07-20 01:00:00,1.991743e+06,58.777990,12.800017,59.538235,23.224394,25.368100,43.078128,12.170469,94.42871,...,100.00000,11.716432,2.086172e+03,59.538235,23.224394,245.68714,144.53843,1,NaN,NaN
1,2018-07-20 01:01:00,1.982156e+03,58.826126,12.796661,59.586994,23.086641,25.493500,43.305633,12.859436,94.42871,...,100.00000,12.249357,2.076584e+03,59.586994,23.086641,246.55198,146.31510,1,0.014414,0.029611
2,2018-07-20 01:02:00,1.982326e+03,58.870080,30553.333333,59.631520,22.950075,25.473633,43.042783,12.578674,94.42871,...,100.00000,12.122468,2.076755e+03,59.631520,22.950075,246.42782,146.14588,1,-0.002069,-0.002820
3,2018-07-20 01:03:00,1.983721e+03,58.909492,12.790008,59.671448,22.814513,25.447811,42.909058,12.373411,94.42871,...,100.00000,11.999847,2.078149e+03,59.671448,22.814513,246.25402,145.82791,1,-0.002897,-0.005300
4,2018-07-20 01:04:00,1.985772e+03,58.944946,12.786683,59.707363,22.680083,25.418883,42.847139,12.210600,94.42871,...,100.00000,11.879883,2.080201e+06,59.707363,22.680083,246.05374,145.42968,1,-0.003338,-0.006637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2018-07-20 04:15:00,1.032485e+03,53.071247,12.241669,53.758750,21.373900,20.827756,35.711119,11.519194,66.10010,...,98.36654,9.681367,1.098585e+03,53.758750,21.373900,216.09112,142.00056,1,0.000460,-0.000853
196,2018-07-20 04:16:00,1.032912e+03,52.738125,12.249908,53.421314,21.432919,20.836183,35.776733,11.573850,66.10010,...,98.14599,9.701203,1.099012e+03,53.421314,21.432919,216.13992,142.01481,1,0.000813,0.000238
197,2018-07-20 04:17:00,1.033215e+03,52.419216,12.258697,53.098274,21.489744,20.845947,35.804719,11.598998,66.10010,...,97.99057,9.714752,1.099315e+03,53.098274,21.489744,216.19764,142.05019,1,0.000962,0.000590
198,2018-07-20 04:18:00,1.033504e+06,52.114690,12.268006,52.789803,21.544283,20.856478,35.820342,11.613708,66.10010,...,97.86097,9.725829,1.099604e+03,52.789803,21.544283,216.26006,142.09179,1,0.001040,0.000693


In [ ]:
# Calcular a Média móvel


janela_media_movel = 1440   # Janela de um dia (1440 minutos)

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'

dados_combinadossim['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = dados_combinadossim['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'

dados_combinadossim['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = dados_combinadossim['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()



In [ ]:
dados_combinadossim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287882 entries, 0 to 287881
Data columns (total 23 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   Datetime                                287882 non-null  datetime64[ns]
 1   RTU_COMP_WATT                           287882 non-null  float64       
 2   RTU_OA_HUM                              287882 non-null  float64       
 3   RTU_OA_TEMP                             287882 non-null  float64       
 4   RTU_RA_HUM                              287882 non-null  float64       
 5   RTU_RA_TEMP                             287882 non-null  float64       
 6   RTU_REFG_COND_TEMP                      287882 non-null  float64       
 7   RTU_REFG_DISC_TEMP                      287882 non-null  float64       
 8   RTU_REFG_SUCT_TEMP                      287882 non-null  float64       
 9   RTU_SA_FAN_WATT                      

In [ ]:
# Calcular o Desvio Padrão


janela_um_dia = 1440 # Defina a janela de 1 dia (1440 minutos)


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
dados_combinadossim['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = dados_combinadossim['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'

dados_combinadossim['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = dados_combinadossim['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()



In [ ]:
# Padronizar Manualmente

# Calcule o Z-Score para cada minuto usando a média e o desvio padrão das colunas
dados_combinadossim['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (dados_combinadossim['Derivada_RTU_REFG_SUCT_PRES'] - dados_combinadossim['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / dados_combinadossim['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']


dados_combinadossim['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (dados_combinadossim['Derivada_RTU_REFG_DISC_PRES'] - dados_combinadossim['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / dados_combinadossim['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']



In [ ]:
# eliminar os valores iniciais Nulos  das primeiras 24 hrs

dados_combinadossim = dados_combinadossim.dropna()

In [ ]:
dados_combinadossim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286442 entries, 1440 to 287881
Data columns (total 27 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   Datetime                                  286442 non-null  datetime64[ns]
 1   RTU_COMP_WATT                             286442 non-null  float64       
 2   RTU_OA_HUM                                286442 non-null  float64       
 3   RTU_OA_TEMP                               286442 non-null  float64       
 4   RTU_RA_HUM                                286442 non-null  float64       
 5   RTU_RA_TEMP                               286442 non-null  float64       
 6   RTU_REFG_COND_TEMP                        286442 non-null  float64       
 7   RTU_REFG_DISC_TEMP                        286442 non-null  float64       
 8   RTU_REFG_SUCT_TEMP                        286442 non-null  float64       
 9   RTU_SA_FAN_W

####dados que serão trabalhados

In [ ]:
#  matriz features sem derivada

X1s= dados_combinadossim.drop(["Target","Datetime", 'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES'], axis =1)
y1s = dados_combinadossim['Target']

In [ ]:
# Matriz  com feature derivada Pura

X2s = dados_combinadossim.drop(["Target","Datetime", 'RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES'], axis =1)
y2s = dados_combinadossim['Target']


In [ ]:
# Matriz com  derivada Média

In [ ]:
X3s= dados_combinadossim.drop(["Target","Datetime", 'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES','RTU_REFG_DISC_PRES','RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES'], axis =1)
y3s = dados_combinadossim['Target']

In [ ]:
# Matriz com  derivada Normalizada

X4s = dados_combinadossim.drop(["Target","Datetime",'RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES',
    'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'], axis =1)
y4s = dados_combinadossim['Target']

In [ ]:
from sklearn.preprocessing import StandardScaler


# Crie um objeto StandardScaler
scalerS = StandardScaler()

# Ajuste e transforme os dados
X1s = scalerS.fit_transform(X1s)
X2s = scalerS.fit_transform(X2s)
X3s = scalerS.fit_transform(X3s)
X4s = scalerS.fit_transform(X4s)

In [ ]:
X_train1s, X_test1s, y_train1s, y_test1s = train_test_split(X1s, y1s, test_size=0.2, stratify=y1s,random_state= 27) # dados sem derivada

X_train2s, X_test2s, y_train2s, y_test2s = train_test_split(X2s, y2s, test_size=0.2, stratify=y2s,random_state= 27) # dados com derivada pura

X_train3s, X_test3s, y_train3s, y_test3s = train_test_split(X3s, y3s, test_size=0.2, stratify=y3s,random_state= 27) # dados com derivada média

X_train4s, X_test4s, y_train4s, y_test4s= train_test_split(X4s, y4s, test_size=0.2, stratify=y4s,random_state= 27) # dados com derivada normalizada


## Modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from statistics import mean
from joblib import dump,load
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
# Criar os modelos
rfs = RandomForestClassifier(random_state=27)
svcs = SVC(random_state=27)
knns = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# Modelo Nos dados reais


# Itere sobre diferentes seleções de features e rótulos
for X, y, label in zip([X_train1, X_train2, X_train3, X_train4], [y_train1, y_train2, y_train3, y_train4], ['Sem_Feature_derivada_R', 'Feature_derivada_pura_R', 'Feature_derivada_Média_R', 'Feature_derivada_Normalizada_R']):


    rfs.fit(X, y)

#Salve o modelo usando joblib
    dump(rfs, f'{label}_model.joblib')





In [ ]:
# Modelo Nos dados simulados


# Itere sobre diferentes seleções de features e rótulos
for X, y, label in zip([X_train1s, X_train2s, X_train3s, X_train4s], [y_train1s, y_train2s, y_train3s, y_train4s], ['Sem_Feature_derivada_S', 'Feature_derivada_pura_S', 'Feature_derivada_Média_S', 'Feature_derivada_Normalizada_S']):


    rfs.fit(X, y)

    # Salve o modelo usando joblib
    dump(rfs, f'{label}_model.joblib')



In [ ]:
lista1 = ['RTU_COMP_WATT','RTU_OA_HUM','RTU_OA_TEMP','RTU_RA_HUM','RTU_RA_TEMP','RTU_REFG_COND_TEMP','RTU_REFG_DISC_TEMP','RTU_REFG_SUCT_TEMP',
          'RTU_SA_FAN_WATT','RTU_SA_FLOW','RTU_SA_HUM','RTU_SA_TEMP','RTU_TOT_WATT','ZA_HUM','ZA_TEMP','RTU_REFG_DISC_PRES','RTU_REFG_SUCT_PRES']

lista2 = ['RTU_COMP_WATT','RTU_OA_HUM','RTU_OA_TEMP','RTU_RA_HUM','RTU_RA_TEMP','RTU_REFG_COND_TEMP','RTU_REFG_DISC_TEMP','RTU_REFG_SUCT_TEMP',
          'RTU_SA_FAN_WATT','RTU_SA_FLOW','RTU_SA_HUM','RTU_SA_TEMP','RTU_TOT_WATT','ZA_HUM','ZA_TEMP','Derivada_RTU_REFG_DISC_PRES','Derivada_RTU_REFG_SUCT_PRES']

lista3 = ['RTU_COMP_WATT','RTU_OA_HUM','RTU_OA_TEMP','RTU_RA_HUM','RTU_RA_TEMP','RTU_REFG_COND_TEMP','RTU_REFG_DISC_TEMP','RTU_REFG_SUCT_TEMP',
          'RTU_SA_FAN_WATT','RTU_SA_FLOW','RTU_SA_HUM','RTU_SA_TEMP','RTU_TOT_WATT','ZA_HUM','ZA_TEMP','MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES']

lista4 = ['RTU_COMP_WATT','RTU_OA_HUM','RTU_OA_TEMP','RTU_RA_HUM','RTU_RA_TEMP','RTU_REFG_COND_TEMP','RTU_REFG_DISC_TEMP','RTU_REFG_SUCT_TEMP',
          'RTU_SA_FAN_WATT','RTU_SA_FLOW','RTU_SA_HUM','RTU_SA_TEMP','RTU_TOT_WATT','ZA_HUM','ZA_TEMP','Z_Score_Derivada_RTU_REFG_SUCT_PRES','Z_Score_Derivada_RTU_REFG_DISC_PRES']

target = ['Target']

In [ ]:
dfx1 = pd.DataFrame(X1, columns=lista1)
dfx1train = pd.DataFrame(X_train1,columns= lista1)
dfx1test = pd.DataFrame(X_test1,columns= lista1)
dfy1 = pd.DataFrame(y1,columns= target)
dfy1train = pd.DataFrame(y_train1,columns= target)
dfy1test = pd.DataFrame(y_test1,columns= target)

In [ ]:
dfx1s = pd.DataFrame(X1s, columns=lista1)
dfx1trains = pd.DataFrame(X_train1s,columns= lista1)
dfx1tests = pd.DataFrame(X_test1s,columns= lista1)
dfy1s = pd.DataFrame(y1s,columns= target)
dfy1trains = pd.DataFrame(y_train1s,columns= target)
dfy1tests = pd.DataFrame(y_test1s,columns= target)

In [ ]:
dfx2 = pd.DataFrame(X2, columns=lista2)
dfx2train = pd.DataFrame(X_train2,columns= lista2)
dfx2test = pd.DataFrame(X_test2,columns= lista2)
dfy2= pd.DataFrame(y2,columns= target)
dfy2train = pd.DataFrame(y_train2,columns= target)
dfy2test = pd.DataFrame(y_test2,columns= target)

In [ ]:
dfx2s = pd.DataFrame(X2s, columns=lista2)
dfx2trains = pd.DataFrame(X_train2s,columns= lista2)
dfx2tests = pd.DataFrame(X_test2s,columns= lista2)
dfy2s = pd.DataFrame(y2s,columns= target)
dfy2trains = pd.DataFrame(y_train2s,columns= target)
dfy2tests = pd.DataFrame(y_test2s,columns= target)

In [ ]:
dfx3 = pd.DataFrame(X3, columns=lista3)
dfx3train = pd.DataFrame(X_train3,columns= lista3)
dfx3test = pd.DataFrame(X_test3,columns= lista3)
dfy3 = pd.DataFrame(y3,columns= target)
dfy3train = pd.DataFrame(y_train3,columns= target)
dfy3test = pd.DataFrame(y_test3,columns= target)

In [ ]:
dfx3s = pd.DataFrame(X3s, columns=lista3)
dfx3trains = pd.DataFrame(X_train3s,columns= lista3)
dfx3tests = pd.DataFrame(X_test3s,columns= lista3)
dfy3s = pd.DataFrame(y3s,columns= target)
dfy3trains = pd.DataFrame(y_train3s,columns= target)
dfy3tests = pd.DataFrame(y_test3s,columns= target)

In [ ]:
dfx4 = pd.DataFrame(X4, columns=lista4)
dfx4train = pd.DataFrame(X_train4,columns= lista4)
dfx4test = pd.DataFrame(X_test4,columns= lista4)
dfy4 = pd.DataFrame(y4,columns= target)
dfy4train = pd.DataFrame(y_train4,columns= target)
dfy4test = pd.DataFrame(y_test4,columns= target)

In [ ]:
dfx4s = pd.DataFrame(X4s, columns=lista4)
dfx4trains = pd.DataFrame(X_train4s,columns= lista4)
dfx4tests = pd.DataFrame(X_test4s,columns= lista4)
dfy4s = pd.DataFrame(y4s,columns= target)
dfy4trains = pd.DataFrame(y_train4s,columns= target)
dfy4tests = pd.DataFrame(y_test4s,columns= target)

In [ ]:
lista_dataframes = ['dfx1','dfx1train', 'dfx1test','dfy1','dfy1train','dfy1test',
                    'dfx1s','dfx1trains', 'dfx1tests','dfy1s','dfy1trains','dfy1tests',
                    'dfx2','dfx2train', 'dfx2test','dfy2','dfy2train','dfy2test',
                    'dfx2s','dfx2trains', 'dfx2tests','dfy2s','dfy2trains','dfy2tests',
                    'dfx3','dfx3train', 'dfx3test','dfy3','dfy3train','dfy3test',
                    'dfx3s','dfx3trains', 'dfx3tests','dfy3s','dfy3trains','dfy3tests',
                    'dfx4','dfx4train', 'dfx4test','dfy4','dfy4train','dfy4test',
                    'dfx4s','dfx4trains', 'dfx4tests','dfy4s','dfy4trains','dfy4tests',
                    ]


In [ ]:
# Salvar o DataFrame em um arquivo CSV

dfx1.to_csv('x1.csv', index=False)
dfx1train.to_csv('x1train.csv', index=False)
dfx1test.to_csv('x1test.csv', index=False)
dfy1.to_csv('y1.csv', index=False)
dfy1train.to_csv('y1train.csv', index=False)
dfy1test.to_csv('y1test.csv', index=False)

dfx1s.to_csv('x1s.csv', index=False)
dfx1trains.to_csv('x1trains.csv', index=False)
dfx1tests.to_csv('x1tests.csv', index=False)
dfy1s.to_csv('y1s.csv', index=False)
dfy1trains.to_csv('y1trains.csv', index=False)
dfy1tests.to_csv('y1tests.csv', index=False)


In [ ]:
# Salvar o DataFrame em um arquivo CSV

dfx2.to_csv('x2.csv', index=False)
dfx2train.to_csv('x2train.csv', index=False)
dfx2test.to_csv('x2test.csv', index=False)
dfy2.to_csv('y2.csv', index=False)
dfy2train.to_csv('y2train.csv', index=False)
dfy2test.to_csv('y2test.csv', index=False)

dfx2s.to_csv('x2s.csv', index=False)
dfx2trains.to_csv('x2trains.csv', index=False)
dfx2tests.to_csv('x2tests.csv', index=False)
dfy2s.to_csv('y2s.csv', index=False)
dfy2trains.to_csv('y2trains.csv', index=False)
dfy2tests.to_csv('y2tests.csv', index=False)

In [ ]:
# Salvar o DataFrame em um arquivo CSV

dfx3.to_csv('x3.csv', index=False)
dfx3train.to_csv('x3train.csv', index=False)
dfx3test.to_csv('x3test.csv', index=False)
dfy3.to_csv('y3.csv', index=False)
dfy3train.to_csv('y3train.csv', index=False)
dfy3test.to_csv('y3test.csv', index=False)

dfx3s.to_csv('x3s.csv', index=False)
dfx3trains.to_csv('x3trains.csv', index=False)
dfx3tests.to_csv('x3tests.csv', index=False)
dfy3s.to_csv('y3s.csv', index=False)
dfy3trains.to_csv('y3trains.csv', index=False)
dfy3tests.to_csv('y3tests.csv', index=False)

In [ ]:
# Salvar o DataFrame em um arquivo CSV

dfx4.to_csv('x4.csv', index=False)
dfx4train.to_csv('x4train.csv', index=False)
dfx4test.to_csv('x4test.csv', index=False)
dfy4.to_csv('y4.csv', index=False)
dfy4train.to_csv('y4train.csv', index=False)
dfy4test.to_csv('y4test.csv', index=False)

dfx4s.to_csv('x4s.csv', index=False)
dfx4trains.to_csv('x4trains.csv', index=False)
dfx4tests.to_csv('x4tests.csv', index=False)
dfy4s.to_csv('y4s.csv', index=False)
dfy4trains.to_csv('y4trains.csv', index=False)
dfy4tests.to_csv('y4tests.csv', index=False)